In [17]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
CV = CountVectorizer()
from scipy.sparse import csr_matrix
from PodcastFeatureEngineering import Podcast_Features
import graphlab as gl
from graphlab import SFrame as sf
import time

In [2]:
client = MongoClient()
# Access/Initiate Database
db = client['Podcast']
# Access/Initiate Table
Podcast_twitter = db['PodcastTwitter_handles']
print Podcast_twitter.count()

153497


In [8]:
gl.set_runtime_config("GRAPHLAB_CACHE_FILE_LOCATIONS", "/mnt/data/")

2016-04-16 20:22:00,385 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1460838119.log


This non-commercial license of GraphLab Create is assigned to crystal.humphries@systemsbiology.org and will expire on November 04, 2016. For commercial licensing options, visit https://dato.com/buy/.


In [3]:
def count_account_with_twitter_handle(pymongo_collection):
    n = 0
    for pod in pymongo_collection.find({}, {'Podcast_Twitter_handle': 1, '_id':0}):
        if 'Podcast_Twitter_handle' in pod and pod['Podcast_Twitter_handle'] != "NaN":
            n+=1
    return n

In [4]:
def return_account_with_twitter_handle(pymongo_collection):
    Podcast_Twitter_handles=[]
    _id=[]
    for pod in pymongo_collection.find({}, {'Podcast_Twitter_handle': 1, '_id':1}):
        if 'Podcast_Twitter_handle' in pod and pod['Podcast_Twitter_handle'] != "NaN":
            Podcast_Twitter_handles.append(pod['Podcast_Twitter_handle'])
            _id.append(pod['_id'])           
    return pd.DataFrame([Podcast_Twitter_handles, _id])

In [ ]:
Podcast_twitter.find_one({}, {'Podcast_Twitter_handle': 1, '_id':1})

In [ ]:
count_account_with_twitter_handle(Podcast_twitter), Podcast_twitter.count()

In [ ]:
new_df = return_account_with_twitter_handle(Podcast_twitter)
new_df.columns = ['Handles', "IDs"]
new_df.to_pickle("twitter_handles_ids_4Mongo.pkl")
new_df.head()

In [5]:
practice = []
n=0
for pod in Podcast_twitter.find({}, {'Title':1, '_id':0, 'TwitterFollowers':1}):
    try:
        if 'TwitterFollowers' in pod and pod['TwitterFollowers'] and pod['TwitterFollowers'][0]!='None':
            practice.append(pod)
            n+=1
    except:
        pass
print n

15819


In [ ]:
import os

In [ ]:
os.environ['PYTHONPATH']

In [6]:
def create_sparse_matrix(list_of_followers):
    list_of_strings = []
    list_of_titles = []
    for ls in list_of_followers:
        x = ' '.join([ str(x) for x in ls['TwitterFollowers']])
        list_of_strings.append(x)
        list_of_titles.append(ls['Title'])
    return list_of_strings, list_of_titles

In [ ]:
%%writefile get_sparsematrix.py

from pymongo import MongoClient
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
CV = CountVectorizer()
from scipy.sparse import csr_matrix
from PodcastFeatureEngineering import Podcast_Features
import graphlab as gl

client = MongoClient()
# Access/Initiate Database
db = client['Podcast']
# Access/Initiate Table
Podcast_twitter = db['PodcastTwitter_handles']
print Podcast_twitter.count()

practice = []
n=0
for pod in Podcast_twitter.find({}, {'Title':1, '_id':0, 'TwitterFollowers':1}):
    try:
        if 'TwitterFollowers' in pod and pod['TwitterFollowers'] and pod['TwitterFollowers'][0]!='None':
            practice.append(pod)
            n+=1
    except:
        pass
print n


def create_sparse_matrix(list_of_followers):
    list_of_strings = []
    list_of_titles = []
    for ls in list_of_followers:
        x = ' '.join([ str(x) for x in ls['TwitterFollowers']])
        list_of_strings.append(x)
        list_of_titles.append(ls['Title'])
    return list_of_strings, list_of_titles


strings,titles = create_sparse_matrix(practice)
final = CV.fit_transform(strings)

a = csr_matrix.sum(final, axis=0)
a = np.array(a).reshape(a.shape[1],)
sparse = final.toarray()
mask = a>1
indices=np.where(mask)[0]
mat = sparse_mat.tocsc()[:,indices]
df = mat.toarray()
df = pd.DataFrame(df)
item_matrix=gl.SFrame(df)

item_matrix.save("IHavebeensaved.gl")

In [ ]:
df = pd.DataFrame(practice)
df.to_pickle("twitter_info.pkl")

In [10]:
strings,titles = create_sparse_matrix(practice)
final = CV.fit_transform(strings)
a = csr_matrix.sum(final, axis=0)
print final.shape
a = np.array(a).reshape(a.shape[1],)
mask = a>1

(15819, 1826440)


In [11]:
u=np.where(mask)[0]

In [12]:
u=np.where(mask)[0]
sparse_mat = final.tocsc()[:,u]
mat = sparse_mat.toarray()

In [13]:
mat = sparse_mat.toarray()

In [14]:
def get_truncated_matrix(mat, max_num):
    df = pd.DataFrame(mat[:,0:max_num])
    df.index = titles
    a = df.sum(axis=1)
    mask2 = a>0
    return df.loc[mask2,:]

In [ ]:
df = pd.DataFrame(mat[:,0:1000])

In [ ]:
sum(mask2)

In [ ]:
df.loc[mask2,:]

In [ ]:
a = df.sum(axis=1)

In [20]:
start = time.time()
attempt = get_truncated_matrix(mat, 1000)
print attempt.shape
end = time.time()
print(end - start)

(2396, 1000)
0.104803085327


In [ ]:
start = time.time()
newnewdf=gl.SFrame(attempt)
end = time.time()
print(end - start)

In [ ]:
newnewdfnewdf.save("sf_50000.gl")

In [ ]:
attempt = get_truncated_matrix(mat, 100000)
print attempt.shape
start = time.time()
newnewdf=gl.SFrame(attempt)
end = time.time()
print(end - start)
newnewdfnewdf.save("sf_100000.gl")

In [ ]:
def get_data(x):
    strings,titles = create_sparse_matrix(x)
    f = CV.fit_transform(strings)
    sf=pd.DataFrame(f.toarray())
    sf = gl.SFrame(sf)
    return sf

In [ ]:
start=0
list_sf = []
for stop in range(1000,11000,1000):
    if stop > len(practice):
        stop = len(practice)
    x = practice[start:stop]
    list_sf.append(get_data(x))
    start = stop

In [ ]:
import time
start = time.time()
x = practice[0:100]
end = time.time()
print(end - start)


In [ ]:
strings,titles = create_sparse_matrix(x)

In [ ]:
f = CV.fit_transform(strings)

In [ ]:
sf=pd.DataFrame(f.toarray())

In [ ]:
sf.head()

In [ ]:
sf = gl.SFrame(sf)
sf.head()

In [ ]:
user_matrix = gl.SFrame(newnewdf)
user_matrix.save("user_twitter_pod.gl") 

In [ ]:
item_matrix = gl.SFrame(newnewdf)

## Creation of a Model

In [ ]:
MFR = gl.recommender.factorization_recommender

In [ ]:
mfr_sd = MFR.create(X_train,
                 item_id='movie_id',
                 target='rating',
                 user_data=user_data,
                 item_data=movie_simple,
                 solver='sgd',
                 linear_regularization=1e-09,
                 regularization=1e-07,
                 num_factors=20)

In [ ]:
PF = Podcast_Features()
df2 = PF.cleaned_df.copy()

In [ ]:
df2 = pd.read_pickle('df2.pkl')

In [ ]:
PF = Podcast_Features(cleaned_df=df2)
df2.to_pickle("df2.pkl")
df2['Subtitle_categories'] = PF.get_new_column_of_text('iTunesSubtitle', n_components=30)
df2.to_pickle("df2.pkl")
df2['Keywords_categories'] = PF.get_new_column_of_text('iTunesKeywords', n_components=30)
df2['Url_Folders'] = df2.NormalizedUrl.apply(lambda x: x.count('/'))
df2.to_pickle("df2.pkl")

In [ ]:
#df2['Summary_categories'] = PF.get_new_column_of_text('iTunesSummary', n_components=30)
#df2.to_pickle("df2.pkl")

#for some reason this data kills the kernel

In [ ]:
dates=[]
for a,b in  zip(df2.DateLastEpisode, df2.DateFirstEpisode):
    try:
        new = (a-b).days
    except:
        new = 'NaT'
    dates.append(new)
dates = np.array(dates)

In [ ]:
df2['dates'] = dates

In [ ]:
g = df2.columns.to_series().groupby(df2.dtypes).groups

In [ ]:
c = g.keys()
c

In [ ]:
df3 = df2.drop(g[c[0]], axis=1)
df3 = df3.drop(g[c[1]], axis=1)
df3['Title'] = df2.Title

In [ ]:
df3.info()

In [ ]:
ISR = gl.recommender.item_similarity_recommender.ItemSimilarityRecommender #SEAFREE

In [ ]:
df2_gl = gl.SFrame(df3)

In [ ]:
df3.

In [ ]:
??gl.item_similarity_recommender.create

In [ ]:
IS = gl.item_similarity_recommender.create(df2_gl, user_id='Title', item_id='CategoryRanking')

In [ ]:
IS.get_similar_users() #(users=['The Moth Podcast'])

In [ ]:
df3.Title